In [4]:
import torch
import torch.nn as nn
from transformers import ViTModel
import matplotlib.pyplot as plt

F:\projects\ResearchProject\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
class ViTDenoiser(nn.Module):
    def __init__(self, pretrained_model='google/vit-base-patch16-224', img_size=224):
        super(ViTDenoiser, self).__init__()
        self.vit = ViTModel.from_pretrained(pretrained_model)
        self.decoder = nn.Sequential(
            nn.Conv2d(768, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 3, kernel_size=3, padding=1)
        )

    def forward(self, noisy_image):
        # Split image into patches and process through ViT
        features = self.vit(noisy_image)['last_hidden_state']
        # Reshape and pass through decoder
        b, n, c = features.size()  # batch, num_patches, channels
        h = w = int(n**0.5)  # Assuming square input
        features = features.transpose(1, 2).view(b, c, h, w)
        denoised_image = self.decoder(features)
        return denoised_image

In [16]:
model = ViTDenoiser()

# Dummy input: Batch of noisy 224x224 RGB images
noisy_input = torch.randn(4, 3, 224, 224)  # Batch of 4 images
denoised_output = model(noisy_input)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: shape '[4, 768, 14, 14]' is invalid for input of size 605184